## 无标记数据的预训练

In [1]:
from importlib.metadata import version

pkgs = ["matplotlib", 
		"numpy", 
		"tiktoken", 
		"torch",
		"tensorflow" # For OpenAI's pretrained weights
		]
for p in pkgs:
	print(f"{p} version: {version(p)}")

import torch
print(torch.__version__)
print(torch.cuda.is_available())

matplotlib version: 3.4.3
numpy version: 1.22.4
tiktoken version: 0.7.0
torch version: 2.4.1
tensorflow version: 2.13.1
2.4.1+cu121
True


### 5.1 Evaluating generative text models

#### 5.1.1 GPT生成文本

In [2]:
# 未有k v cache

import torch
from previous_chapters import GPTModel

GPT_CONFIG_124M = {
	"vocab_size": 50257,   # Vocabulary size
	"context_length": 256, # Shortened context length (orig: 1024)
	"emb_dim": 768,        # Embedding dimension
	"n_heads": 12,         # Number of attention heads
	"n_layers": 12,        # Number of layers
	"drop_rate": 0.1,      # Dropout rate
	"qkv_bias": False      # Query-key-value bias
}

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.eval();  # Disable dropout during inference	# 全局影响 可影响其他cell model.train()

In [3]:
import tiktoken
from previous_chapters import generate_text_simple

def text_to_token_ids(text, tokenizer):
	encoded = tokenizer.encode(text, allowed_special={"<|endoftext|"})
	encoded_tensor = torch.tensor(encoded).unsqueeze(0)
	return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
	flat = token_ids.squeeze(0)
	return tokenizer.decode(flat.tolist())

start_context = "Every effort moves you"
tokenizer = tiktoken.get_encoding("gpt2")

token_ids = generate_text_simple(
	model=model,
	idx=text_to_token_ids(start_context, tokenizer),
	max_new_tokens=10,
	context_size=GPT_CONFIG_124M["context_length"]
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

Output text:
 Every effort moves you rentingetic wasnم refres RexMeCHicular stren


#### 5.1.2 loss 交叉熵 perplexity

In [4]:
inputs = torch.tensor([[16833, 3626, 6100],   # ["every effort moves",
						[40,    1107, 588]])   #  "I really like"]

targets = torch.tensor([[3626, 6100, 345  ],  # [" effort moves you",
						[1107,  588, 11311]]) #  " really like chocolate"]

In [5]:
# 通过model + softmax -> 概率
with torch.no_grad():
	logits = model(inputs)

probas = torch.softmax(logits, dim=-1)
print(probas.shape)
# 每个batch 中的 context_length 对应的50257为词表大小 其数值为对应该词的概率

torch.Size([2, 3, 50257])


- 将概率转回文本

In [6]:
token_ids = torch.argmax(probas, dim=-1, keepdim=True)
print("Token IDs:\n", token_ids)

Token IDs:
 tensor([[[16657],
         [  339],
         [42826]],

        [[49906],
         [29669],
         [41751]]])


In [7]:
print(f"Inputs batch 1: {token_ids_to_text(inputs[0], tokenizer)}")
print(f"Targets batch 1: {token_ids_to_text(targets[0], tokenizer)}")
print(f"Outputs batch 1: {token_ids_to_text(token_ids[0].flatten(), tokenizer)}")

Inputs batch 1: every effort moves
Targets batch 1:  effort moves you
Outputs batch 1:  Armed heNetflix


In [8]:
text_idx = 0
target_probas_1 = probas[text_idx, [0, 1, 2], targets[text_idx]]
# target_probas_1 = probas[text_idx, :, targets[text_idx]]	# :会进行广播 导致结果为3*3 对每个词表概率都取了targets[text_idx]的三个值

print("Text 1:", target_probas_1)

text_idx = 1
target_probas_2 = probas[text_idx, [0, 1, 2], targets[text_idx]]
print("Text 2:", target_probas_2)

a = probas[text_idx]
print(a.shape)
b = a[[0,1,2]]
print(b.shape)
c = a[:]
print(c.shape)

Text 1: tensor([7.4540e-05, 3.1061e-05, 1.1563e-05])
Text 2: tensor([1.0337e-05, 5.6776e-05, 4.7559e-06])
torch.Size([3, 50257])
torch.Size([3, 50257])
torch.Size([3, 50257])


- 目标是令以上三值达到1

In [9]:
# 取对数
log_probas = torch.log(torch.cat((target_probas_1, target_probas_2)))
print(log_probas)

# 求平均
avg_log_probas = torch.mean(log_probas)
print(avg_log_probas)

tensor([ -9.5042, -10.3796, -11.3677, -11.4798,  -9.7764, -12.2561])
tensor(-10.7940)


- 目标变为 令平均对数的值尽可能的大 上限为0

- *-1 变为尽可能小 


In [10]:
neg_avg_log_probas = avg_log_probas * -1
print(neg_avg_log_probas)

tensor(10.7940)


- 交叉熵 流程

model+softmax求概率 $\rightarrow$ 取出目标的概率 $\rightarrow$ 取对数 $\rightarrow$ 求平均 $\rightarrow$ *-1

In [11]:
# Logits have shape (batch_size, num_tokens, vocab_size)
print("Logits shape:", logits.shape)

# Targets have shape (batch_size, num_tokens)
print("Targets shape:", targets.shape)

Logits shape: torch.Size([2, 3, 50257])
Targets shape: torch.Size([2, 3])


In [12]:
logits_flat = logits.flatten(0, 1)
targets_flat = targets.flatten()

print("Flattened logits:", logits_flat.shape)
print("Flattened targets:", targets_flat.shape)

Flattened logits: torch.Size([6, 50257])
Flattened targets: torch.Size([6])


In [13]:
loss = torch.nn.functional.cross_entropy(logits_flat, targets_flat)
print(loss)

tensor(10.7940)


- 困惑度 exponential

- perplexity = exp(交叉熵)

In [14]:
perplexity = torch.exp(loss)
print(perplexity)

tensor(48725.8203)


#### 5.1.3 training validation set loss

In [15]:
import os
import urllib.request

file_path = "../ch02/the-verdict.txt"
url = "https://raw.githubusercontent.com/rasbt/LLMs-from-scratch/main/ch02/01_main-chapter-code/the-verdict.txt"

if not os.path.exists(file_path):
	with urllib.request.urlopen(url) as response:
		text_data = response.read().decode('utf-8')
	with open(file_path, "w", encoding="utf-8") as file:
		file.write(text_data)
else:
	with open(file_path, "r", encoding="utf-8") as file:
		text_data = file.read()

In [16]:
# First 99 characters
print(text_data[:99])
# Last 99 characters
print(text_data[-99:])

I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 
it for me! The Strouds stand alone, and happen once--but there's no exterminating our kind of art."


In [17]:
total_characters = len(text_data)
total_tokens = len(tokenizer.encode(text_data))

print("Characters:", total_characters)
print("Tokens:", total_tokens)

Characters: 20479
Tokens: 5145


In [18]:
from previous_chapters import create_dataloader_v1

train_ratio = 0.90
split_idx = int(train_ratio * len(text_data))
train_data = text_data[:split_idx]
val_data = text_data[split_idx:]

torch.manual_seed(123)           # 设置CPU随机种子
torch.cuda.manual_seed(123)      # 设置当前GPU随机种子
torch.cuda.manual_seed_all(123)  # 设置所有GPU随机种子（多卡时）

train_loader = create_dataloader_v1(
	train_data,
	batch_size=2,
	max_length=GPT_CONFIG_124M["context_length"],
	stride=GPT_CONFIG_124M["context_length"],
	drop_last=True,
	shuffle=True,
	num_workers=0
)

val_loader = create_dataloader_v1(
    val_data,
    batch_size=2,
    max_length=GPT_CONFIG_124M["context_length"],
    stride=GPT_CONFIG_124M["context_length"],
    drop_last=False,
    shuffle=False,
    num_workers=0
)

In [19]:
# Sanity check

if total_tokens * (train_ratio) < GPT_CONFIG_124M["context_length"]:
    print("Not enough tokens for the training loader. "
          "Try to lower the `GPT_CONFIG_124M['context_length']` or "
          "increase the `training_ratio`")

if total_tokens * (1-train_ratio) < GPT_CONFIG_124M["context_length"]:
    print("Not enough tokens for the validation loader. "
          "Try to lower the `GPT_CONFIG_124M['context_length']` or "
          "decrease the `training_ratio`")

In [20]:
print("Train loader:")
for x, y in train_loader:
    print(x.shape, y.shape)

print("\nValidation loader:")
for x, y in val_loader:
    print(x.shape, y.shape)

Train loader:
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])

Validation loader:
torch.Size([2, 256]) torch.Size([2, 256])


In [21]:
train_tokens = 0
for input_batch, target_batch in train_loader:
    train_tokens += input_batch.numel()

val_tokens = 0
for input_batch, target_batch in val_loader:
    val_tokens += input_batch.numel()

print("Training tokens:", train_tokens)
print("Validation tokens:", val_tokens)
print("All tokens:", train_tokens + val_tokens)

Training tokens: 4608
Validation tokens: 512
All tokens: 5120


In [22]:
def calc_loss_batch(input_batch, target_batch, model, device):
	"""
	计算给定批次的交叉熵
	"""
	input_batch, target_batch = input_batch.to(device), target_batch.to(device)
	logits = model(input_batch)
	loss = torch.nn.functional.cross_entropy(logits.flatten(0, 1), target_batch.flatten())
	return loss

def calc_loss_loader(data_loader, model, device, num_batches=None):
	total_loss=0.
	if len(data_loader) == 0:
		return float("nan")
	
	if num_batches is None:
		num_batches = len(data_loader)
	else:	# 防止batch的量比所有数量都多
		num_batches = min(num_batches, len(data_loader))
	
	for i, (input_batch, target_batch) in enumerate(data_loader):
		if i >= num_batches:
			break

		loss = calc_loss_batch(input_batch, target_batch, model, device)
		total_loss += loss.item()

	return total_loss / num_batches

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model.to(device)

with torch.no_grad():
	train_loss = calc_loss_loader(train_loader, model, device)
	val_loss = calc_loss_loader(val_loader, model, device)

print("Training loss:", train_loss)
print("Validation loss:", val_loss)

Training loss: 10.98758273654514
Validation loss: 10.981104850769043


### 5.2 Training a LLM

In [32]:
def train_model_simple(model, train_loader, val_loader, optimizer, device, num_epochs, 
						eval_freq, eval_iter, start_context, tokenizer):
	train_losses, val_losses, track_tokens_seen = [], [], []
	tokens_seen, global_step = 0, -1

	for epoch in range(num_epochs):
		model.train()

		for input_batch, target_batch in train_loader:
			optimizer.zero_grad()
			loss = calc_loss_batch(input_batch, target_batch, model, device)
			loss.backward()
			optimizer.step()
			tokens_seen += input_batch.numel()	# 已训练的token计数
			global_step += 1

			# Optional evaluation step
			if global_step % eval_freq == 0:
				train_loss, val_loss = evaluate_model(
					model, train_loader, val_loader, device, eval_iter
				)
				train_losses.append(train_loss)
				val_losses.append(val_loss)
				track_tokens_seen.append(tokens_seen)
				print(f"Ep {epoch+1} (Step {global_step:06d}): "
						f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")
	
		# 每个epoch之后打印示例文本
		generate_and_print_sample(
			model, tokenizer, device, start_context
		)

	return train_losses, val_losses, track_tokens_seen

def evaluate_model(model, train_loader, val_loader, device, eval_iter):
	model.eval()
	with torch.no_grad():
		train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
		val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
	model.train()
	return train_loss, val_loss

def generate_and_print_sample(model, tokenizer, device, start_context):
	model.eval()
	context_size = model.pos_emb.weight.shape[0]
	encoded = text_to_token_ids(start_context, tokenizer).to(device)
	with torch.no_grad():
		token_ids = generate_text_simple(
			model=model, idx=encoded, max_new_tokens=50, context_size=context_size
		)
	decoded_text = token_ids_to_text(token_ids, tokenizer)
	print(decoded_text.replace("\n", "<|/n|>"))
	model.train()

In [31]:
import time
start_time = time.time()

torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)
model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=0.0004, weight_decay=0.1)

num_epochs = 10
train_losses, val_losses, tokens_seen = train_model_simple(
    model, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=5, eval_iter=5,
    start_context="Every effort moves you", tokenizer=tokenizer
)

end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

Ep 1 (Step 000000): Train loss 9.818, Val loss 9.928
Ep 1 (Step 000005): Train loss 8.065, Val loss 8.335
Every effort moves you,,,,,,,,,,,,.





































Ep 2 (Step 000010): Train loss 6.622, Val loss 7.051
Ep 2 (Step 000015): Train loss 6.047, Val loss 6.600
Every effort moves you, and,, and,,,,,,, and,.



































Ep 3 (Step 000020): Train loss 5.586, Val loss 6.477
Ep 3 (Step 000025): Train loss 5.523, Val loss 6.399
Every effort moves you, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and, and
Ep 4 (Step 000030): Train loss 5.128, Val loss 6.366
Ep 4 (Step 000035): Train loss 4.941, Val loss 6.366
Every effort moves you a a to the a. Gisburn, and a. Gisburn. I had the of the of the of the of the of the of the of the of the of the of the of the of the of the. I had a
Ep 5 (Step 000040): Train loss 4.340, Val loss 6.246
Every effort moves you, with a, in the of the pi